https://spark.apache.org/docs/3.3.2/rdd-programming-guide.html

In [1]:
from pyspark import SparkContext, SparkConf
import os
from pyspark.sql.session import SparkSession

spark = SparkSession \
    .builder \
    .appName('Python Spark Postgresql') \
    .config("spark.jars", "./postgresql-42.6.0.jar") \
    .config('spark.driver.extraClassPath', './postgresql-42.6.0.jar') \
    .getOrCreate()

23/03/30 15:23:38 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/03/30 15:23:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/30 15:23:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.format("jdbc") \
  .option("url", "jdbc:postgresql://localhost/hadoop?user=hadoop") \
  .option("dbtable", "gameclicks") \
  .load()

In [3]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- clickid: integer (nullable = true)
 |-- userid: integer (nullable = true)
 |-- usersessionid: integer (nullable = true)
 |-- ishit: integer (nullable = true)
 |-- teamid: integer (nullable = true)
 |-- teamlevel: integer (nullable = true)



In [4]:
df.count()

755806

In [5]:
df.show(5)

+-------------------+-------+------+-------------+-----+------+---------+
|          timestamp|clickid|userid|usersessionid|ishit|teamid|teamlevel|
+-------------------+-------+------+-------------+-----+------+---------+
|2016-05-26 15:06:55|    105|  1038|         5916|    0|    25|        1|
|2016-05-26 15:07:09|    154|  1099|         5898|    0|    44|        1|
|2016-05-26 15:07:14|    229|   899|         5757|    0|    71|        1|
|2016-05-26 15:07:14|    322|  2197|         5854|    0|    99|        1|
|2016-05-26 15:07:20|     22|  1362|         5739|    0|    13|        1|
+-------------------+-------+------+-------------+-----+------+---------+
only showing top 5 rows



In [6]:
df.select("userid", "teamlevel").show(5)

+------+---------+
|userid|teamlevel|
+------+---------+
|  1038|        1|
|  1099|        1|
|   899|        1|
|  2197|        1|
|  1362|        1|
+------+---------+
only showing top 5 rows



In [7]:
df.filter(df["teamlevel"] > 1)

DataFrame[timestamp: timestamp, clickid: int, userid: int, usersessionid: int, ishit: int, teamid: int, teamlevel: int]

In [8]:
df.filter(df["teamlevel"] > 1).show(5)

+-------------------+-------+------+-------------+-----+------+---------+
|          timestamp|clickid|userid|usersessionid|ishit|teamid|teamlevel|
+-------------------+-------+------+-------------+-----+------+---------+
|2016-05-27 23:06:57|  26994|  1513|         7550|    0|    13|        2|
|2016-05-27 23:07:04|  27084|   868|         7625|    0|    35|        2|
|2016-05-27 23:07:06|  27029|  1453|         7593|    0|    22|        2|
|2016-05-27 23:07:09|  27287|  1282|         7826|    0|    93|        2|
|2016-05-27 23:07:12|  27195|  1473|         7774|    0|    75|        2|
+-------------------+-------+------+-------------+-----+------+---------+
only showing top 5 rows



In [8]:
df.filter(df["teamlevel"] > 1).select("userid", "teamlevel").show(5)

+------+---------+
|userid|teamlevel|
+------+---------+
|  1513|        2|
|   868|        2|
|  1453|        2|
|  1282|        2|
|  1473|        2|
+------+---------+
only showing top 5 rows



In [7]:
df.groupBy("ishit").count().show()

+-----+------+
|ishit| count|
+-----+------+
|    1| 83383|
|    0|672423|
+-----+------+



In [10]:
df.select("userid", "teamlevel").show(5)

+------+---------+
|userid|teamlevel|
+------+---------+
|  1038|        1|
|  1099|        1|
|   899|        1|
|  2197|        1|
|  1362|        1|
+------+---------+
only showing top 5 rows



In [8]:
from pyspark.sql.functions import *
df.select(mean('ishit'), sum('ishit')).show()


+------------------+----------+
|        avg(ishit)|sum(ishit)|
+------------------+----------+
|0.1103232840173272|     83383|
+------------------+----------+



In [10]:
df2 = spark.read.format("jdbc") \
  .option("url", "jdbc:postgresql://localhost/hadoop?user=hadoop") \
  .option("dbtable", "adclicks") \
  .load()

In [11]:
df2.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- txid: integer (nullable = true)
 |-- usersessionid: integer (nullable = true)
 |-- teamid: integer (nullable = true)
 |-- userid: integer (nullable = true)
 |-- adid: integer (nullable = true)
 |-- adcategory: string (nullable = true)



In [12]:
merge = df.join(df2, 'userid')

In [13]:
merge.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- clickid: integer (nullable = true)
 |-- usersessionid: integer (nullable = true)
 |-- ishit: integer (nullable = true)
 |-- teamid: integer (nullable = true)
 |-- teamlevel: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- txid: integer (nullable = true)
 |-- usersessionid: integer (nullable = true)
 |-- teamid: integer (nullable = true)
 |-- adid: integer (nullable = true)
 |-- adcategory: string (nullable = true)



In [14]:
merge.show(5)

+------+-------------------+-------+-------------+-----+------+---------+-------------------+-----+-------------+------+----+----------+
|userid|          timestamp|clickid|usersessionid|ishit|teamid|teamlevel|          timestamp| txid|usersessionid|teamid|adid|adcategory|
+------+-------------------+-------+-------------+-----+------+---------+-------------------+-----+-------------+------+----+----------+
|   471|2016-05-26 15:12:28|    109|         5863|    0|    27|        1|2016-06-15 07:40:58|37570|        34330|    27|   6|    movies|
|   471|2016-05-26 15:12:28|    109|         5863|    0|    27|        1|2016-06-15 00:10:58|37103|        34330|    27|  23|   fashion|
|   471|2016-05-26 15:12:28|    109|         5863|    0|    27|        1|2016-06-14 13:06:53|36168|        34330|    27|  26|    movies|
|   471|2016-05-26 15:12:28|    109|         5863|    0|    27|        1|2016-06-14 09:31:41|35833|        34330|    27|  12| computers|
|   471|2016-05-26 15:12:28|    109|     